In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install pandas
pip install sqlite3
pip install numpy

In [ ]:
import pandas as pd
import sqlite3
import numpy as np
import os

# --- 1. SETUP: SIMULATE GEE DATA DOWNLOAD ---
# In a real project, you would replace this with:
df_raw = pd.read_csv('Mangrove_Health_TimeSeries_FIXED.csv')


# --- 2. DIMENSION TABLE GENERATION ---

# 2.1 DIM_TIME
def create_dim_time(df):
    """Generates the Dim_Time table from the unique acquisition dates."""
    df_time = df[['Date_Acquired']].drop_duplicates().copy()
    df_time['Date_Acquired'] = pd.to_datetime(df_time['Date_Acquired'])

    df_time['Year'] = df_time['Date_Acquired'].dt.year
    df_time['Month'] = df_time['Date_Acquired'].dt.month
    df_time['Day_of_Year'] = df_time['Date_Acquired'].dt.dayofyear

    # Simple seasonal classification (adjust for local monsoon dates if needed)
    def classify_season(month):
        if 6 <= month <= 9: return 'Monsoon'
        if month == 10 or month == 11: return 'Post-Monsoon'
        if 1 <= month <= 5 or month == 12: return 'Dry'
        return 'Other'

    df_time['Season'] = df_time['Month'].apply(classify_season)
    df_time['Date_ID'] = (df_time['Date_Acquired'].dt.strftime('%Y%m%d')).astype(int)

    # Select final columns and set Date_ID as the Primary Key
    df_time = df_time[['Date_ID', 'Date_Acquired', 'Year', 'Month', 'Day_of_Year', 'Season']]
    return df_time.rename(columns={'Date_Acquired': 'Full_Date'})

# 2.2 DIM_LOCATION
def create_dim_location(df):
    """Generates the Dim_Location table from unique spatial attributes."""
    # Location is uniquely defined by Lat/Lon and Subregion
    df_location = df[['AOI_Subregion', 'Longitude', 'Latitude']].drop_duplicates().copy()
    df_location.reset_index(drop=True, inplace=True)
    df_location['Location_ID'] = df_location.index + 1 # Simple integer PK

    # Placeholder for Shoreline Proximity (would be derived via GIS buffer analysis)
    df_location['Shoreline_Proximity'] = np.select(
        [df_location['AOI_Subregion'] == 'Panvel',
         df_location['AOI_Subregion'] == 'Uran'],
        ['Interior', 'Fringe'],
        default='Interior'
    )

    df_location = df_location[['Location_ID', 'Longitude', 'Latitude', 'AOI_Subregion', 'Shoreline_Proximity']]
    return df_location

# 2.3 DIM_SENSOR
def create_dim_sensor():
    """Generates the Dim_Sensor table (Static)."""
    data = {
        'Sensor_ID': [1],
        'Satellite_Name': ['Sentinel-2A'],
        'Resolution_m': [10],
        'Data_Source': ['GEE/COPERNICUS/S2_SR_HARMONIZED']
    }
    df_sensor = pd.DataFrame(data)
    return df_sensor


# --- 3. FACT TABLE GENERATION ---
def create_fact_table(df_raw, df_time, df_location, df_sensor):
    """Merges Dimensions back into the Raw data to create the Fact Table."""
    df_fact = df_raw.copy()
    df_fact.rename(columns={'Date_Acquired': 'Full_Date'}, inplace=True)

    # Merge Dim_Time (to get Date_ID)
    df_fact = pd.merge(df_fact, df_time[['Full_Date', 'Date_ID']], on='Full_Date', how='left')

    # Merge Dim_Location (to get Location_ID)
    df_fact = pd.merge(df_fact, df_location[['Location_ID', 'AOI_Subregion', 'Longitude', 'Latitude']],
                       on=['AOI_Subregion', 'Longitude', 'Latitude'], how='left')

    # Assign Sensor_ID (assuming all data comes from Sentinel-2A/2B mix, use Sensor_ID=1 as primary key)
    # Note: In a real scenario, the GEE export would specify which satellite acquired the image.
    df_fact['Sensor_ID'] = 1

    # Create the final Fact Table structure
    df_fact['Mangrove_Flag'] = (df_fact['NDVI_Value'] > 0.4).astype(int)

    df_fact = df_fact[[
        'Date_ID', 'Location_ID', 'Sensor_ID',
        'NDVI_Value', 'EVI_Value', 'Mangrove_Flag'
        # Other measures like NDVI_Change are calculated separately on aggregate data
    ]]

    df_fact.reset_index(drop=True, inplace=True)
    df_fact['Fact_ID'] = df_fact.index + 1

    return df_fact

# --- 4. EXECUTION AND DATABASE CREATION ---
def build_star_schema_db(db_name='mangrove_analysis.db'):
    """Main function to build the Star Schema and store in SQLite."""

    if os.path.exists(db_name):
        os.remove(db_name)

    # 1. Prepare DataFrames
    df_raw = create_sample_gee_data()
    df_time = create_dim_time(df_raw)
    df_location = create_dim_location(df_raw)
    df_sensor = create_dim_sensor()
    df_fact = create_fact_table(df_raw, df_time, df_location, df_sensor)

    # 2. Connect to SQLite Database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # 3. Write DataFrames to Database (OLAP Load)
    df_time.to_sql('Dim_Time', conn, if_exists='replace', index=False)
    df_location.to_sql('Dim_Location', conn, if_exists='replace', index=False)
    df_sensor.to_sql('Dim_Sensor', conn, if_exists='replace', index=False)
    df_fact.to_sql('Fact_Mangrove_Health', conn, if_exists='replace', index=False)

    # 4. Verification and Example Query (OLAP Query)
    print("\n--- Star Schema Built Successfully ---")
    print(f"Database created: {db_name}")
    print("\nExample OLAP Query (Mean NDVI by Subregion and Year):")

    query = """
    SELECT
        T.Year,
        L.AOI_Subregion,
        AVG(F.NDVI_Value) AS Avg_NDVI
    FROM
        Fact_Mangrove_Health F
    JOIN
        Dim_Time T ON F.Date_ID = T.Date_ID
    JOIN
        Dim_Location L ON F.Location_ID = L.Location_ID
    GROUP BY
        T.Year, L.AOI_Subregion
    ORDER BY
        L.AOI_Subregion, T.Year;
    """

    results = pd.read_sql_query(query, conn)
    print(results.to_string())

    conn.close()

if __name__ == '__main__':
    build_star_schema_db()


Creating sample GEE data...

--- Star Schema Built Successfully ---
Database created: mangrove_analysis.db

Example OLAP Query (Mean NDVI by Subregion and Year):
    Year AOI_Subregion  Avg_NDVI
0   2018        Panvel  0.695700
1   2019        Panvel  0.687250
2   2020        Panvel  0.663750
3   2021        Panvel  0.662033
4   2022        Panvel  0.632567
5   2023        Panvel  0.616967
6   2024        Panvel  0.609500
7   2018         Thane  0.699250
8   2019         Thane  0.676850
9   2020         Thane  0.659900
10  2021         Thane  0.659667
11  2022         Thane  0.636850
12  2023         Thane  0.633750
13  2024         Thane  0.616467
14  2018          Uran  0.691033
15  2019          Uran  0.675417
16  2020          Uran  0.671250
17  2021          Uran  0.646183
18  2022          Uran  0.640417
19  2023          Uran  0.629100
20  2024          Uran  0.614583


/tmp/ipython-input-4251160162.py:13: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(start='2018-01-01', end='2024-12-31', freq='2M')


In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

# --- CONFIGURATION ---
DB_NAME = 'mangrove_analysis.db'
CSV_IN_NAME = 'Mangrove_Health_TimeSeries_FIXED.csv' # Using the fixed GEE file name
ARFF_OUT_NAME = 'mangrove_weka_analysis_FLAT.arff'

# --- ARFF EXPORT FUNCTION (New) ---

def create_arff_string(df, relation_name="mangrove_health_analysis"):
    """Converts a Pandas DataFrame into a Weka-compatible ARFF string."""
    arff_content = []

    # 1. Relation Declaration
    arff_content.append(f"@relation {relation_name}\n")

    # 2. Attribute Declarations
    for col in df.columns:
        series = df[col]

        if col in ['NDVI_Value', 'EVI_Value']:
            # Explicitly set health indices as numeric
            arff_content.append(f"@attribute {col} numeric")
        elif series.dtype == 'object' or col in ['Year', 'Month', 'Season', 'AOI_Subregion', 'Shoreline_Proximity']:
            # Nominal (Categorical) attributes
            unique_values = series.dropna().unique()

            # Sort year and month numerically, others alphabetically
            if col in ['Year', 'Month']:
                 unique_values = sorted([str(int(x)) for x in unique_values])
            else:
                 unique_values = sorted([str(x) for x in unique_values])

            # Format nominal list: {val1, val2, val3,...}
            nominal_list = '{' + ','.join(unique_values) + '}'
            arff_content.append(f"@attribute {col} {nominal_list}")
        else:
            # All other columns (like internal IDs, if any remain)
            arff_content.append(f"@attribute {col} string")

    arff_content.append("\n@data")

    # 3. Data Section
    # Convert DataFrame to list of records (rows)
    for index, row in df.iterrows():
        row_values = []
        for val in row:
            if pd.isnull(val):
                row_values.append('?') # Weka missing value syntax
            else:
                # Remove commas from nominal data to prevent Weka errors
                row_values.append(str(val).replace(',', ''))
        arff_content.append(','.join(row_values))

    return "\n".join(arff_content)


# --- 1. DATA PREPARATION AND DIMENSION CREATION FUNCTIONS ---

def create_dim_time(df):
    """Generates the Dim_Time table from the unique acquisition dates."""
    df_time = df[['Date_Acquired']].drop_duplicates().copy()
    df_time['Date_Acquired'] = pd.to_datetime(df_time['Date_Acquired'])

    df_time['Year'] = df_time['Date_Acquired'].dt.year.astype(str)
    df_time['Month'] = df_time['Date_Acquired'].dt.month.astype(str)

    def classify_season(month):
        if 6 <= month <= 9: return 'Monsoon'
        if month == 10 or month == 11: return 'Post_Monsoon'
        return 'Dry'

    # Ensure month is integer for comparison before converting to nominal string
    df_time['Season'] = df_time['Month'].astype(int).apply(classify_season)

    df_time = df_time[['Date_Acquired', 'Year', 'Month', 'Season']]
    return df_time.rename(columns={'Date_Acquired': 'Full_Date'})

def create_dim_location(df):
    """Generates the Dim_Location table, simulating the spatial breakdown (AOI Subregions)."""
    # NOTE: AOI_Subregion is necessary for the OLAP Slice/Dice/Drill-down
    # Assign subregions randomly for the time series data points.
    subregions = ['Panvel', 'Uran', 'Thane_Creek']
    df['AOI_Subregion'] = np.random.choice(subregions, size=len(df))

    df_location = df[['AOI_Subregion']].drop_duplicates().copy()
    df_location.reset_index(drop=True, inplace=True)

    df_location['Shoreline_Proximity'] = np.select(
        [df_location['AOI_Subregion'] == 'Uran',
         df_location['AOI_Subregion'] == 'Thane_Creek'],
        ['Fringe', 'Interior'],
        default='Delta'
    )
    return df_location

def export_weka_data():
    """Builds the Star Schema structure and exports a flat table as ARFF for Weka."""
    print(f"Starting Weka Data Export Process. Reading: {CSV_IN_NAME}")

    if not os.path.exists(CSV_IN_NAME):
        print(f"ERROR: Cannot find {CSV_IN_NAME}. Please download the FIXED GEE CSV.")
        return

    # Load the real GEE data
    df_raw = pd.read_csv(CSV_IN_NAME)

    # --- CRITICAL FIXES FOR CSV COLUMNS ---
    # 1. Rename the primary GEE Date column to the internal standard 'Date_Acquired'
    df_raw.rename(columns={'Date': 'Date_Acquired'}, inplace=True)

    # 2. Clean up date formats and convert the time column
    df_raw['Date_Acquired'] = pd.to_datetime(df_raw['Date_Acquired']).dt.normalize()

    # 3. Rename the Index columns to standard names for clarity in ARFF
    df_raw.rename(columns={'NDVI': 'NDVI_Value', 'EVI': 'EVI_Value'}, inplace=True)

    # Check for empty file content
    if df_raw.empty or df_raw['NDVI_Value'].isnull().all():
        print("ERROR: CSV file is empty or contains no valid data. Please check the GEE export.")
        return

    # --- Create Dimensions and Flatten ---
    df_time = create_dim_time(df_raw)
    df_location = create_dim_location(df_raw)

    # Merge Time Dimension
    df_weka = pd.merge(df_raw, df_time[['Full_Date', 'Year', 'Month', 'Season']],
                       left_on='Date_Acquired', right_on='Full_Date', how='left')

    # Merge Location Dimension (relies on AOI_Subregion simulation in create_dim_location)
    df_weka = pd.merge(df_weka, df_location[['AOI_Subregion', 'Shoreline_Proximity']],
                       on='AOI_Subregion', how='left')

    # Select the final attributes Weka needs (dropping spectral bands, coordinates, and redundant time fields)
    df_weka = df_weka[[
        'Year', 'Month', 'Season', 'AOI_Subregion', 'Shoreline_Proximity',
        'NDVI_Value', 'EVI_Value'
    ]].dropna().copy()

    # Final cleanup before ARFF export
    df_weka['Year'] = df_weka['Year'].astype(str)
    df_weka['Month'] = df_weka['Month'].astype(str)

    # 4. Export to ARFF
    arff_string = create_arff_string(df_weka)
    with open(ARFF_OUT_NAME, 'w') as f:
        f.write(arff_string)

    print("\n--- Python Export Successful ---")
    print(f"Flattened Weka ARFF file saved as: {ARFF_OUT_NAME}")
    print(f"Total records exported: {len(df_weka)}")
    print("\nFile is ready for Weka Explorer (Clustering & Association Tabs).")

# Execute the process
if __name__ == '__main__':
    export_weka_data()


Starting Weka Data Export Process. Reading: Mangrove_Health_TimeSeries_FIXED.csv

--- Python Export Successful ---
Flattened Weka ARFF file saved as: mangrove_weka_analysis_FLAT.arff
Total records exported: 214

File is ready for Weka Explorer (Clustering & Association Tabs).
